In [ ]:
import numpy as np

import sys
sys.path.append("..")
import solax as sx

## 1. Basis

In [ ]:
basis = sx.Basis(["1100", "1010", "1001", "0110", "0101", "0011"])

In [ ]:
print(basis)

In [ ]:
with sx.dets_printing_limit(10):
    print(basis)

In [ ]:
print(basis.bitlen)

In [ ]:
print(len(basis))

In [ ]:
print(basis[0])

In [ ]:
print(basis[0:3])

In [ ]:
print(basis[0, 1, 4])

In [ ]:
print(basis[True, True, False, False, True, False])

In [ ]:
print(basis == basis[::-1])

In [ ]:
basis_even = basis[::2]
basis_odd = basis[1::2]

In [ ]:
print(basis_even + basis_odd == basis)

In [ ]:
print(basis + basis == basis)

In [ ]:
print(basis % basis_even == basis_odd)

## 2. State

In [ ]:
state = sx.State(basis, np.ones(6))

In [ ]:
print(state)

In [ ]:
print(state.basis)

In [ ]:
print(state.coeffs)

In [ ]:
state_odd = state % basis_even
print(state_odd)

In [ ]:
print(state_odd.basis == basis_odd)

In [ ]:
print(2 * state)

In [ ]:
print(state * 1j)

In [ ]:
print(state / 2)

In [ ]:
print(-state)

In [ ]:
state1 = state[0, 1, 4]
print(state1)

In [ ]:
state2 = state[0, 3]
print(state2)

In [ ]:
print(state1 + state2)

In [ ]:
print(state1 - state2)

In [ ]:
print(0.1 + 0.1 - 0.2)
print(0.1 + 0.2 - 0.3)

In [ ]:
print(state1 * state2)

In [ ]:
print(state * state)

In [ ]:
state_normalized = state.normalize()
print(state_normalized * state_normalized)

In [ ]:
state3 = state2[-1]
state123 = state1 - state2 - state3
print(state123)

In [ ]:
state_chopped1 = state123.chop(1e-14)
print(state_chopped1)

In [ ]:
state_chopped2 = state123.chop(1.5)
print(state_chopped2)

In [ ]:
state_chopped3 = state123.chop(2.5)
print(state_chopped3)
print(len(state_chopped3))

In [ ]:
state_mini = state[:2]
state_a = 0.1 * state_mini + 0.2 * state_mini
print(state_a)

In [ ]:
state_b = 0.3 * state_mini
print(state_b)

In [ ]:
state_diff = state_a - state_b
print(state_diff)

In [ ]:
state_zero = state_diff.chop(1e-14)
print(len(state_zero))

## 3. OperatorTerm

In [ ]:
daggers = (1, 0)

posits = np.array([
    [1, 3],
    [2, 4]
]) - 1

coeffs = np.array([
    1.0,
    1.0
])

In [ ]:
V0 = sx.OperatorTerm(daggers, posits, coeffs)
print(V0)

In [ ]:
print(V0.hconj)

In [ ]:
V = V0 + V0.hconj
print(V)

In [ ]:
basis = sx.Basis(["1001", "0110"])
coeffs = np.array([1.0, -1.0])

psi = sx.State(basis, coeffs)
psi = psi.normalize()

print(psi)

In [ ]:
result_psi = V(psi)
print(result_psi)

In [ ]:
result_basis = V(psi.basis)
print(result_basis)

In [ ]:
result_psi_batches = V(psi, det_batch_size=1, op_batch_size=2)

In [ ]:
s = result_psi_batches - result_psi
s = s.chop(1e-14)
print(len(s))

## 4. Operator

In [ ]:
op = sx.Operator(daggers, posits, coeffs)
print(op)

In [ ]:
print(sx.Operator(V0))

In [ ]:
daggers_u = (1, 0, 1, 0)
posits_u = np.array([
    [1, 1, 2, 2],
    [3, 3, 4, 4]
]) - 1
coeffs_u = np.array([0.25, 0.75])

U = sx.Operator(daggers_u, posits_u, coeffs_u)
print(U)

In [ ]:
H = 1 + V0 + V0.hconj + U
print(H)

In [ ]:
print(H[1, 0, 1, 0])

In [ ]:
print(H["scalar"])

In [ ]:
for key in H.keys():
    print(key)

In [ ]:
print(len(H))

In [ ]:
for key, term in H.items():
    if key != "scalar":
        print(f"Length of the OperatorTerm {key} is {len(term)}")

In [ ]:
H_without_V = H.drop(1, 0)
print((1, 0) in H_without_V)

In [ ]:
H_without_scalar = H.drop("scalar")
print("scalar" in H_without_scalar)

In [ ]:
H_chopped1 = H.chop((1, 0, 1, 0), 0.5)
print(len(H_chopped1[1, 0, 1, 0]))

In [ ]:
H_chopped2 = H.chop((1, 0, 1, 0), 1.0)
print((1, 0, 1, 0) in H_chopped2)

## 5. OperatorMatrix

In [ ]:
basis = sx.Basis(["1001", "1100", "0110", "0011"])

In [ ]:
matrix = H.build_matrix(basis)
print(matrix)

In [ ]:
print(matrix.size)

In [ ]:
print(matrix.num_nonzero)

In [ ]:
coo_matrix = matrix.to_scipy()

In [ ]:
dense_matrix = coo_matrix.todense()
print(dense_matrix)

In [ ]:
def print_matrix(m):
    print(m.to_scipy().todense())

In [ ]:
print_matrix(matrix)

In [ ]:
print_matrix(
    matrix.displace(2, 1)
)

In [ ]:
print_matrix(
    matrix.displace(-1, -1)
)

In [ ]:
print_matrix(
    matrix.window((1, 1), (3, 4))
)

In [ ]:
sub_basis = basis[0, 2]
print(sub_basis)

In [ ]:
print_matrix(
    matrix.shrink_basis(basis, sub_basis)
)

In [ ]:
print_matrix(
    matrix.chop(1.1)
)

In [ ]:
matrix3_3 = matrix.displace(-1, -1)
print_matrix(matrix3_3)

In [ ]:
matrix5_2 = matrix.displace(1, -2)
print_matrix(matrix5_2)

In [ ]:
print_matrix(
    matrix3_3 + matrix5_2
)

In [ ]:
m1 = matrix.window((0, 0), (2, 2))
print_matrix(m1)

In [ ]:
m2 = matrix.shrink_basis(basis, basis[:2])
print_matrix(m2)

In [ ]:
print(
    (m1 - m2).chop(1e-14).num_nonzero == 0
)

In [ ]:
print(m1.size == m2.size)